In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 93.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.0 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
import pandas as pd
from keras.optimizers import Adam
import google.generativeai as genai
genai.configure(api_key='your_key_here')

generation_config = {
  "temperature":0,
  "top_p":1,
  "top_k":1,
  "max_output_tokens":2000
}

safety_settings = [
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_NONE"
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_NONE"
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_NONE"
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_NONE"
    },
]

model_gen = genai.GenerativeModel('gemini-pro', generation_config=generation_config, safety_settings=safety_settings)

class_names = ['apple_pie',
                'baby_back_ribs',
                'baklava',
                'beef_carpaccio',
                'beef_tartare',
                'beet_salad',
                'beignets',
                'bibimbap',
                'bread_pudding',
                'breakfast_burrito',
                'bruschetta',
                'caesar_salad',
                'cannoli',
                'caprese_salad',
                'carrot_cake',
                'ceviche',
                'cheesecake',
                'cheese_plate',
                'chicken_curry',
                'chicken_quesadilla',
                'chicken_wings',
                'chocolate_cake',
                'chocolate_mousse',
                'churros',
                'clam_chowder',
                'club_sandwich',
                'crab_cakes',
                'creme_brulee',
                'croque_madame',
                'cup_cakes',
                'deviled_eggs',
                'donuts',
                'dumplings',
                'edamame',
                'eggs_benedict',
                'escargots',
                'falafel',
                'filet_mignon',
                'fish_and_chips',
                'foie_gras',
                'french_fries',
                'french_onion_soup',
                'french_toast',
                'fried_calamari',
                'fried_rice',
                'frozen_yogurt',
                'garlic_bread',
                'gnocchi',
                'greek_salad',
                'grilled_cheese_sandwich',
                'grilled_salmon',
                'guacamole',
                'gyoza',
                'hamburger',
                'hot_and_sour_soup',
                'hot_dog',
                'huevos_rancheros',
                'hummus',
                'ice_cream',
                'lasagna',
                'lobster_bisque',
                'lobster_roll_sandwich',
                'macaroni_and_cheese',
                'macarons',
                'miso_soup',
                'mussels',
                'nachos',
                'omelette',
                'onion_rings',
                'oysters',
                'pad_thai',
                'paella',
                'pancakes',
                'panna_cotta',
                'peking_duck',
                'pho',
                'pizza',
                'pork_chop',
                'poutine',
                'prime_rib',
                'pulled_pork_sandwich',
                'ramen',
                'ravioli',
                'red_velvet_cake',
                'risotto',
                'samosa',
                'sashimi',
                'scallops',
                'seaweed_salad',
                'shrimp_and_grits',
                'spaghetti_bolognese',
                'spaghetti_carbonara',
                'spring_rolls',
                'steak',
                'strawberry_shortcake',
                'sushi',
                'tacos',
                'takoyaki',
                'tiramisu',
                'tuna_tartare',
                'waffles']

def preprocess(image, shape=224, scale=False):
    image = tf.image.decode_image(image, channels=3)
    image = tf.image.resize(image, size=([shape, shape]))
    if scale:
        image = image/255.
    return image

@st.cache
def predicting(image, model):
    image = preprocess(image, scale=False)
    # image = tf.cast(tf.expand_dims(image, axis=0), tf.int16)
    pred_conf = model.predict(tf.expand_dims(image, axis=0))
    pred_class = class_names[pred_conf.argmax()]

    return pred_class, pred_conf

st.set_page_config(page_title="Food Identification")

# Sidebar
st.sidebar.title("What is Food Identification?")
st.sidebar.write("""
Food Identification is used to identify the food in your image using a CNN model for classification.

Transfer Learning is used, with the pre-trained model being EfficientNetB1.

The dataset used for training is Food101, which has 101 different classes of food.

Created by Jai Ramani, Aditya Pune and Raj Ranadive.
""")

# Main Body of Page
st.title("Food Identification")
file = st.file_uploader(label="Upload an image of food.",
                        type=["jpg", "jpeg", "png"])

model = tf.keras.models.load_model("/content/drive/MyDrive/food_v5.hdf5")

if not file:
    st.stop()

else:
    image = file.read()
    st.image(image, use_column_width=True)

pred_button = st.checkbox("Predict and Generate Recipe")

if pred_button:
    pred_class, pred_conf = predicting(image, model)
    st.session_state['class'] = pred_class
    pred_class = pred_class.replace("_"," ").capitalize()
    st.subheader(f'The food has been identified as {pred_class}')
    st.success(f'Confidence: {pred_conf.max()*100:.2f}%', icon="✅")

    specif = st.radio('Any customizations?', ['Yes','No'])

    if specif == 'Yes':
        specification = st.text_input("Enter your customization")
        if len(specification)>1:
            content = "Give me a recipe for " + st.session_state['class'] + " with customization: " + specification
            response = model_gen.generate_content(content)
            for candidate in response.candidates:
                text = [part.text for part in candidate.content.parts]
            st.write(text[0])

    else:
        content = "Give me a recipe for " + st.session_state['class']
        response = model_gen.generate_content(content)
        for candidate in response.candidates:
            text = [part.text for part in candidate.content.parts]
        st.write(text[0])

Writing app.py


In [ ]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 2.108s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit fix` to fix them, or `npm audit` for details


In [ ]:
!streamlit run /content/app.py --client.showErrorDetails=false &>/content/logs.txt &

In [ ]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

In [ ]:
!npx localtunnel --port 8501